In [26]:
import csv
import string
import numpy as np
import math

In [2]:
words = {}
wordIndex = 0
punctuations = string.punctuation + '\n'

In [3]:
def getProperSentence(sentence):
    returnSentence = sentence.lower()
    replace_punctuation = string.maketrans(punctuations, ' '*len(punctuations))
    returnSentence = returnSentence.translate(replace_punctuation)
    return returnSentence

In [4]:
messageList = []
cleanedMessageList = []

In [5]:
def populateMessageList():
    with open('train.csv','rb') as train:
        reader = csv.reader(train)
        for row in reader:
            message =row[4]
            messageList.append(message)
            
populateMessageList()

def populateCleanedMessageList():
    with open('train.csv','rb') as train:
        reader = csv.reader(train)
        for row in reader:
            message =row[4]
            cleanedMessageList.append(getProperSentence(message))
            
populateCleanedMessageList()

In [6]:
wordsIndex = {}
wordCount = 0
wordCountForIdf = {}

In [9]:
def populateWordsIndex():
    global wordCount
    for message in cleanedMessageList:
        for word in message.split(' '):
            if not word.isdigit():
                if word not in wordsIndex:
                    wordsIndex[word] = wordCount
                    wordCount += 1

populateWordsIndex()

In [13]:
def populateWordCountForIdf():
    for word in wordsIndex:
        count = 0
        for message in cleanedMessageList:
            if word in message:
                count += 1
        wordCountForIdf[word] = count
        
populateWordCountForIdf()

In [43]:
def tf(givenWord, message): #Assuming cleaned message
    count = 0
    for word in message.split(' '):
        if givenWord == word:
            count += 1.0 
    return count

def idf(givenWord):
    N = len(messageList)
    n = wordCountForIdf[givenWord]
    return math.log(N*1.0/(1+n))

In [57]:
def makeFeatureVector(sentence):
    tempSen = sentence
    retVal = np.zeros(wordCount)
    for word in tempSen.split(' '):
        if word.isdigit():
            continue
        indexOfWord = wordsIndex[word]
        retVal[indexOfWord] = tf(word,tempSen)*idf(word)
    return retVal

In [61]:
def returnFeatureSet():
    index = 0
    ret = np.array([])
    for message in cleanedMessageList:        
        featureVector = makeFeatureVector(message)
#             print featureVector
        if index == 0:
            ret = np.hstack((ret,featureVector))
            index = index + 1
        else:
            ret = np.vstack((ret, featureVector))
    return ret

In [62]:
featureVectorSet = returnFeatureSet()

In [64]:
from sklearn.cluster import KMeans

In [65]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(featureVectorSet)

In [66]:
def getClusters(clusterIndices, messageList, requiredIndex):
    retVal = []
    for i in range(len(clusterIndices)):
        if clusterIndices[i] == requiredIndex:
            retVal.append(messageList[i])
    return retVal

In [76]:
getClusters(kmeans.labels_,messageList,10)

[]